In [11]:
#Reinforment learing
%matplotlib inline


In [8]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import VecVideoRecorder
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

Environment

In [33]:
env = gym.make('CartPole-v0')

In [34]:
episodes = 5
from pathlib import Path

log_dir= "runs/",

for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    reward = 0 
    
    while not done:
        env.render() # Display the image
        action = env.action_space.sample()
        state, r, done, info= env.step(action)
        reward+=r
    print('Episode:{} Score:{}'.format(episode, reward))
env.close()

Episode:1 Score:18.0
Episode:2 Score:12.0
Episode:3 Score:18.0
Episode:4 Score:19.0
Episode:5 Score:45.0


In [7]:
log_path = os.path.join('trainings','logs')
log_path

'trainings/logs'

creating and training the RL model

In [4]:
env = gym.make('CartPole-v1')
env = DummyVecEnv([lambda: env])
m = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)
m.learn(total_timesteps=10000)


/home/badhon/miniconda3/envs/sofa/lib/python3.9/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device
Logging to trainings/logs/PPO_6
-----------------------------
| time/              |      |
|    fps             | 1251 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1013        |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008687853 |
|    clip_fraction        | 0.0837      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00767     |
|    learning_rate        | 0.0003      |
|    loss                 | 7.64        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.012      |
|    value_loss           | 49.9        |
---------------------------

In [38]:
model_path = os.path.join('trainings','models','CartPole')

In [39]:
m.save(model_path)

In [40]:
del m

In [41]:
m = PPO.load(model_path, env=env)

In [42]:
m

Evaluation

In [13]:
evaluate_policy(m, env, n_eval_episodes=20, render = True)

(394.95, 104.77951851387752)

In [45]:
env.close()

Test Model

In [14]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    reward = 0 
    
    while not done:
        env.render() # Display the image
        action, _ = m.predict(obs)
        obs, r, done, info= env.step(action)
        reward+=r
    print('Episode:{} Score:{}'.format(episode, reward))
env.close()

Episode:1 Score:[191.]
Episode:2 Score:[102.]
Episode:3 Score:[106.]
Episode:4 Score:[160.]
Episode:5 Score:[182.]


Tensor Board Logs

In [6]:
t_log_path = os.path.join(log_path,'PPO_2')
t_log_path

'trainings/logs/PPO_2'

In [ ]:
#!tensorboard --logdir=(t_log_path)

callback for training stage

In [16]:
save_model = os.path.join('trainings','models')

stop_callback = StopTrainingOnRewardThreshold(reward_threshold=180, verbose=1)
eval_callback = EvalCallback(env, callback_on_new_best = stop_callback, eval_freq=10000, best_model_save_path=save_model, verbose=1)

In [17]:
model = PPO('MlpPolicy',env,verbose=1, tensorboard_log= log_path)

Using cpu device


In [18]:
model.learn(total_timesteps =20000, callback = eval_callback)

Logging to trainings/logs/PPO_8
-----------------------------
| time/              |      |
|    fps             | 1744 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1300        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008561747 |
|    clip_fraction        | 0.0956      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00196    |
|    learning_rate        | 0.0003      |
|    loss                 | 5.56        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0147     |
|    value_loss           | 52.1        |
-----------------------------------------
--

Chaning the policy

In [21]:
net_arch = [dict(pi = [128,128], vf=[128,128])]

In [22]:
model = PPO('MlpPolicy',env,verbose=1, tensorboard_log= log_path, policy_kwargs={'net_arch':net_arch})

Using cpu device


/home/badhon/miniconda3/envs/sofa/lib/python3.9/site-packages/stable_baselines3/common/policies.py:460: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


In [24]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to trainings/logs/PPO_10
-----------------------------
| time/              |      |
|    fps             | 1653 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 1241         |
|    iterations           | 2            |
|    time_elapsed         | 3            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0072872033 |
|    clip_fraction        | 0.0651       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.59        |
|    explained_variance   | 0.494        |
|    learning_rate        | 0.0003       |
|    loss                 | 22.7         |
|    n_updates            | 50           |
|    policy_gradient_loss | -0.0134      |
|    value_loss           | 49.5         |
--------------------------